In [1]:
import os
import requests
from datetime import datetime

import geopandas as gpd
import geojsonio

import pandas as pd
from pandas.io.json import json_normalize

import folium
from folium import plugins

## USGS GeoJSON format

schema = {
  type: "FeatureCollection",
  metadata: {
    generated: Long Integer,
    url: String,
    title: String,
    api: String,
    count: Integer,
    status: Integer
  },
  bbox: [
    minimum longitude,
    minimum latitude,
    minimum depth,
    maximum longitude,
    maximum latitude,
    maximum depth
  ],
  features: [
    {
      type: "Feature",
      properties: {
        mag: Decimal,
        place: String,
        time: Long Integer,
        updated: Long Integer,
        tz: Integer,
        url: String,
        detail: String,
        felt:Integer,
        cdi: Decimal,
        mmi: Decimal,
        alert: String,
        status: String,
        tsunami: Integer,
        sig:Integer,
        net: String,
        code: String,
        ids: String,
        sources: String,
        types: String,
        nst: Integer,
        dmin: Decimal,
        rms: Decimal,
        gap: Decimal,
        magType: String,
        type: String
      },
      geometry: {
        type: "Point",
        coordinates: [
          longitude,
          latitude,
          depth
        ]
      },
      id: String
    },
    …
  ]
}

## Getting data

In [2]:
#data = requests.get("https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_day.geojson")
data = requests.get("https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_week.geojson")


jsondata = data.json()
quakes = pd.json_normalize(jsondata['features'])

In [3]:
quakes.head(2)

,type,id,properties.mag,properties.place,properties.time,properties.updated,properties.tz,properties.url,properties.detail,properties.felt,...,properties.types,properties.nst,properties.dmin,properties.rms,properties.gap,properties.magType,properties.type,properties.title,geometry.type,geometry.coordinates
0,Feature,us7000dftf,5.4,Kermadec Islands region,1614912590335,1614913863292,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,...,",losspager,origin,phase-data,shakemap,",NaN,11.630,1.16,45.0,mb,earthquake,M 5.4 - Kermadec Islands region,Point,"[-176.59, -28.3296, 10]"
1,Feature,us7000dftb,5.3,Kermadec Islands region,1614912192756,1614913263040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,...,",origin,phase-data,",NaN,11.628,1.21,59.0,mb,earthquake,M 5.3 - Kermadec Islands region,Point,"[-176.5986, -28.3314, 10]"


## Cleaning the data

Some of these fields are essientially useless in all cases. A good example is geometry.type which according to the schema definition is going to always be point.

A couple are just redundant. For example 'title' is just 'mag' plus 'place'

Some are probably useless but *might* be useful, particularly if we apply a ML model to it later. So, I'll keep them for now. But the prepending everything with 'properties' or 'geometry' -- that has to go.

In [4]:
quakes.columns = quakes.columns.str.replace('properties.', "", regex=False)
quakes.columns = quakes.columns.str.replace('geometry.', "", regex=False)
quakes.drop(['id', 'type', 'updated', 'tz', 'mmi', 'detail', 'felt','cdi', 'felt', 'types', 'nst', 'type', 'title'], 
        axis=1, inplace=True)

In [5]:
quakes.head(2)

,mag,place,time,url,alert,status,tsunami,sig,net,code,ids,sources,dmin,rms,gap,magType,coordinates
0,5.4,Kermadec Islands region,1614912590335,https://earthquake.usgs.gov/earthquakes/eventp...,green,reviewed,0,449,us,7000dftf,",us7000dftf,",",us,",11.630,1.16,45.0,mb,"[-176.59, -28.3296, 10]"
1,5.3,Kermadec Islands region,1614912192756,https://earthquake.usgs.gov/earthquakes/eventp...,None,reviewed,0,432,us,7000dftb,",us7000dftb,",",us,",11.628,1.21,59.0,mb,"[-176.5986, -28.3314, 10]"


In [6]:
quakes['ids'] = quakes['ids'].str.strip(',')
quakes['sources'] = quakes['sources'].str.strip(',')

In [7]:
quakes['time'] = pd.to_datetime(quakes['time'], unit='ms')


In [8]:
quakes

,mag,place,time,url,alert,status,tsunami,sig,net,code,ids,sources,dmin,rms,gap,magType,coordinates
0,5.40,Kermadec Islands region,2021-03-05 02:49:50.335,https://earthquake.usgs.gov/earthquakes/eventp...,green,reviewed,0,449,us,7000dftf,us7000dftf,us,11.6300,1.1600,45.00,mb,"[-176.59, -28.3296, 10]"
1,5.30,Kermadec Islands region,2021-03-05 02:43:12.756,https://earthquake.usgs.gov/earthquakes/eventp...,None,reviewed,0,432,us,7000dftb,us7000dftb,us,11.6280,1.2100,59.00,mb,"[-176.5986, -28.3314, 10]"
2,5.80,Kermadec Islands region,2021-03-05 02:30:35.314,https://earthquake.usgs.gov/earthquakes/eventp...,green,reviewed,0,518,us,7000dft1,us7000dft1,us,9.8050,1.0500,41.00,mb,"[-176.5468, -28.7301, 10]"
3,2.70,"63 km NW of Arctic Village, Alaska",2021-03-05 02:28:52.611,https://earthquake.usgs.gov/earthquakes/eventp...,None,automatic,0,112,ak,0212xvrb0l,ak0212xvrb0l,ak,NaN,0.7800,NaN,ml,"[-146.5566, 68.5583, 1.7]"
4,2.64,"12km S of Mariposa, CA",2021-03-05 02:11:48.470,https://earthquake.usgs.gov/earthquakes/eventp...,None,automatic,0,109,nc,73531396,nc73531396,nc,0.1939,0.3100,69.00,md,"[-119.982666, 37.3818321, 17.09]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,4.70,"230 km ENE of Levuka, Fiji",2021-02-26 06:22:36.381,https://earthquake.usgs.gov/earthquakes/eventp...,None,reviewed,0,340,us,7000ddty,us7000ddty,us,3.2250,0.7300,74.00,mb,"[-178.5694, -17.5654, 550.47]"
340,2.80,"24 km S of Mina, Nevada",2021-02-26 06:03:35.892,https://earthquake.usgs.gov/earthquakes/eventp...,None,reviewed,0,121,nn,00801264,"us7000ddtg,nn00801264","us,nn",0.0320,0.1749,150.09,ml,"[-118.0903, 38.169, 3.6]"
341,2.80,"25 km S of Mina, Nevada",2021-02-26 06:03:30.605,https://earthquake.usgs.gov/earthquakes/eventp...,None,reviewed,0,121,nn,00801310,"us7000ddyx,nn00801310","us,nn",0.0290,0.1019,76.88,ml,"[-118.0884, 38.1634, 4.8]"
342,2.98,"5 km WSW of Tallaboa, Puerto Rico",2021-02-26 04:43:39.790,https://earthquake.usgs.gov/earthquakes/eventp...,None,reviewed,0,137,pr,2021057002,"us7000ddta,pr2021057002","us,pr",0.1205,0.0900,180.00,md,"[-66.7588, 17.9705, 7]"


## Parsing the data

Breakout the coords column

In [9]:
quakes['longitude'] = quakes.coordinates.str[0]
quakes['latitude'] = quakes.coordinates.str[1]
quakes['depth'] = quakes.coordinates.str[2]

quakes.drop(['coordinates'], axis=1, inplace=True)

In [10]:
quakes['datetime'] = pd.to_datetime(quakes['time']).dt.strftime("%Y-%m-%d %H:%M")
quakes.drop(['time'], axis=1, inplace=True)

In [11]:
quakes.round({'longitude': 5, 'latitude': 5, 'depth': 0})
quakes.head(2)

,mag,place,url,alert,status,tsunami,sig,net,code,ids,sources,dmin,rms,gap,magType,longitude,latitude,depth,datetime
0,5.4,Kermadec Islands region,https://earthquake.usgs.gov/earthquakes/eventp...,green,reviewed,0,449,us,7000dftf,us7000dftf,us,11.630,1.16,45.0,mb,-176.5900,-28.3296,10.0,2021-03-05 02:49
1,5.3,Kermadec Islands region,https://earthquake.usgs.gov/earthquakes/eventp...,None,reviewed,0,432,us,7000dftb,us7000dftb,us,11.628,1.21,59.0,mb,-176.5986,-28.3314,10.0,2021-03-05 02:43


In [12]:
m = folium.Map(
    location=[0, 0],
    tiles='cartodbpositron',
    zoom_start=1.8
)

Figure out how to get time to display

In [13]:
fmtr = "function(num) {return L.Util.formatNum(num, 3);};"
folium.plugins.MousePosition(separator=' / ', prefix="Lat/Long: ", lat_formatter=fmtr, lng_formatter=fmtr).add_to(m)

In [14]:
for i, row in quakes.iterrows():
    folium.CircleMarker((row.latitude, row.longitude), radius=row.mag*2.2, 
                        color='red', weight = 0, opacity =.4, fill=True, fill_color='orange', fill_opacity=.3, 
                        popup=[f"Time: {row.datetime},\n Mag: {row.mag},\n Depth: {row.depth} km"]).add_to(m)
    

In [15]:
boundaries = ('data/PB2002_boundaries.json')
line_style = {'color': '#FF3333', 'weight': 1, 'opacity':row.mag*1.4}


folium.GeoJson(
    boundaries,
    name='major fault lines',
    style_function=lambda x:line_style,
    smooth_factor=4.0,
      
).add_to(m)
m